In [6]:
%matplotlib qt5
import mne
import numpy as np
import pandas as pd
mne.__version__

'0.18.2'

In [7]:
mne.set_log_level("WARNING")

In [12]:
data = np.loadtable("CANlabStudy/p300/110_eeg.csv", delimiter=",")

AttributeError: module 'numpy' has no attribute 'loadtable'